In [11]:
import numpy as np
import time

# Sender Program

In [1]:
def read_file(file):
    with open(file,'r') as f:
        s=f.read()
    return s

# Insert Header and Trailer

# Get the mac address by using  ipconfig /all in command prompt

In [14]:
def make_frame(data,divisor):
    source='6018952B925C'#'0000'*5+'0001'#000001
    dest='6018952B925C'#'0000'*5+'0010'#000010
    length='0000000000000001'#=0001
    frame_seq_no='00000001'#=01
    trailer=crc(data,divisor)
    header=source+dest+length+frame_seq_no
    frame=header+data+trailer
    return frame

# Calculate Checksum

In [4]:
def checksum(s,clip):
    l=[]
    s='0'*(len(s)%clip)+s
    for i in range(0,len(s),clip):
        l.append(s[i:i+clip])
    #print(l)
    decimal=list(map(lambda x:int(x,2),l))
    s_dec=sum(decimal)
    #print(s_dec)
    bin_s_dec=str(bin(s_dec)[2:])
    
    clip_val=int(bin_s_dec[len(bin_s_dec)-clip:],2)
    #print(clip_val)
    clip_val_before=int(bin_s_dec[:len(bin_s_dec)-clip],2)
    
    clip_sum=clip_val+clip_val_before
    clip_sum_bin=str(bin(clip_sum)[2:])
    if(clip-len(clip_sum_bin)>0):
        clip_sum_bin='0'*(clip-len(clip_sum_bin))+clip_sum_bin
    clip_sum_bin_complement = int(''.join('1' if bit == '0' else '0' for bit in clip_sum_bin),2)
    decimal.append(clip_sum_bin_complement)
    return decimal#BOOM This is the checksum

In [5]:
print(checksum('01111011110000000110',4))

[7, 11, 12, 0, 6, 9]


# Calculate CRC

In [9]:
def crc(s, divisor):
    orig_s=s
    s += '0' * (len(divisor) - 1)
    s = list(s)
    orig_divisor=divisor
    divisor = list(divisor)
    for i in range(len(s) - len(divisor) + 1):
        if s[i] == '1':  # Only perform XOR if the bit is 1
            for j in range(len(divisor)):
                s[i + j] = str(int(s[i + j]) ^ int(divisor[j]))
    remainder = ''.join(s[-(len(divisor) - 1):])
    
    return remainder

In [10]:
print(crc('1001','1011'))

110


# Insert Random Delay

In [12]:
def cal_delay(max_delay):
    return np.random.randint(0,max_delay)

# 0=>Single Bit, 1=>Two Isolated Single Bit, 2=>Odd number of errors, 3=> Burst Errors

In [13]:
import numpy as np
import time
def error_inject(s, etype=4):
    np.random.seed(int(time.time()))
    #etype=np.random.randint(0, 8)changed
    print(etype)
    esize=np.random.randint(0,len(s))
    s = list(s)  
    
    if etype == 0:
        np.random.seed(int(time.time()))
        ebit = np.random.randint(0, len(s))
        s[ebit] = '1' if s[ebit] == '0' else '0'
    
    elif etype == 1:
        np.random.seed(int(time.time()))
        ebit1 = np.random.randint(0, len(s))
        ebit2 = np.random.randint(0, len(s))
        while ebit1 == ebit2 or abs(ebit1 - ebit2) == 1:
            ebit1 = np.random.randint(0, len(s))
            ebit2 = np.random.randint(0, len(s))
        s[ebit1] = '1' if s[ebit1] == '0' else '0'
        s[ebit2] = '1' if s[ebit2] == '0' else '0'
    
    elif etype == 2:
        np.random.seed(int(time.time()))
        mid = len(s) // 2
        n_error = np.random.randint(0, mid) * 2 + 1
        random_list = np.random.choice(len(s), n_error, replace=False)
        for idx in random_list:
            s[idx] = '1' if s[idx] == '0' else '0'
    
    elif etype == 3:
        np.random.seed(int(time.time()))
        random_list = np.random.choice(len(s), esize, replace=False)
        print(random_list)
        for idx in random_list:
            s[idx] = '1' if s[idx] == '0' else '0'
    else:
        pass
    s[3]='0'
    s[19]='1'
    return ''.join(s)  


# First get s from text file, then call error_inject then call either crc or checksum then send the codeword along with the divisor or checksum as applicable

In [11]:
import socket
import time
import pickle

In [12]:
def send_binary_string(filename, choice,host, port):
    s=read_file(filename)
    s_error=error_inject(s,1)
    #augword=checksum or remainder(in case of crc)
    if choice==1:#checksum
        clip=int(input('Enter clip size'))
        augword=checksum(s,clip)
        serialized_data=pickle.dumps((s_error,clip,augword,choice))
    else:
        divisor=input('Enter divisor')
        augword=crc(s,divisor)
        serialized_data = pickle.dumps((s_error,divisor,augword,choice))
    # Create a socket object
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        # Connect to the receiver
        s.connect((host, port))
        
        # Send the serialized data
        s.sendall(serialized_data)

In [39]:
if __name__ == '__main__':
    # Change these to your server's address and port
    HOST = 'localhost'
    PORT = 65432
    filename='C:/Users/sudip/PycharmProjects/pythonProject1/My Python Programs/DSA and OOPs and Machine Learning/bin_data.txt'
    choice=int(input('Enter choice'))
    send_binary_string(filename,choice,HOST,PORT)

Enter choice 1


1


Enter clip size 4
